In [ ]:
import os
import re
import json
import yaml
import pandas as pd
import psycopg2
from openai import OpenAI
from dotenv import load_dotenv

# Load .env
load_dotenv()

# Lấy thông tin kết nối từ biến môi trường
DB_CONFIG = {
    "dbname": os.getenv("DBNAME") or os.getenv("POSTGRES_DB"),
    "user": os.getenv("DBUSER") or os.getenv("POSTGRES_USER"),
    "password": os.getenv("DBPASSWORD") or os.getenv("POSTGRES_PASSWORD"),
    "host": os.getenv("DBHOST") or "localhost",
    "port": int(os.getenv("DBPORT", 5432)),
    "sslmode": os.getenv("SSL_MODE", "require")
}

# Thiết lập OpenAI API
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def connect_to_database():
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        print("✅ Kết nối thành công đến PostgreSQL.")
        return conn
    except Exception as e:
        print(f"❌ Lỗi kết nối cơ sở dữ liệu: {e}")
        return None

def get_schema_and_samples(conn):
    try:
        cursor = conn.cursor()
        cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema='public';")
        tables = cursor.fetchall()
        schema_info = {}
        for (table,) in tables:
            cursor.execute(f"SELECT column_name, data_type FROM information_schema.columns WHERE table_name = '{table}';")
            schema_info[table] = [{"column_name": col, "data_type": dtype} for col, dtype in cursor.fetchall()]
            cursor.execute(f"SELECT * FROM {table} LIMIT 3;")
            sample_rows = cursor.fetchall()
            colnames = [desc[0] for desc in cursor.description]
            schema_info[f"{table}_samples"] = [dict(zip(colnames, row)) for row in sample_rows]
        cursor.close()
        return schema_info
    except Exception as e:
        return {"error": str(e)}

def load_metadata():
    try:
        with open("metadata.yml", "r") as file:
            return yaml.safe_load(file)
    except FileNotFoundError:
        return {}

def generate_sql_query(user_question, schema_info=None):
    prompt = f"""
You are a PostgreSQL expert working with a single table called `stocks`.

This table contains daily stock information with the following columns:

- Date: Date of the record (format: YYYY-MM-DD)
- Price: The closing price of the stock on that date (FLOAT)
- Open: The opening price of the stock on that date (FLOAT)
- High: The highest price of the stock on that date (FLOAT)
- Low: The lowest price of the stock on that date (FLOAT)
- Vol.: Trading volume (format: float + 'M' suffix for millions)
- Change %: Daily percentage change in stock price (can be positive or negative)

Assume this table is already in a PostgreSQL database as `stocks`.

User Question:
{user_question}

Write a correct and optimized PostgreSQL query to answer this question. 
Do not explain or wrap the query in markdown. Just return raw SQL.
    """

    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}]
        )
        sql_query = response.choices[0].message.content
        sql_query = re.sub(r'^```sql', '', sql_query).strip('` \n')
        return sql_query
    except Exception as e:
        print(f"❌ Lỗi sinh SQL: {e}")
        return None

def execute_sql_query(conn, query):
    try:
        df = pd.read_sql(query, conn)
        return df
    except Exception as e:
        print(f"❌ Lỗi thực thi SQL: {e}")
        return None
    
def run_chat(question: str):
    conn = connect_to_database()
    if conn is None:
        return

    schema_info = get_schema_and_samples(conn)
    sql_query = generate_sql_query(question, schema_info)

    if not sql_query:
        print("⚠️ Không thể sinh truy vấn SQL.")
        return

    print(f"\n🧠 SQL sinh ra:\n{sql_query}")
    results = execute_sql_query(conn, sql_query)

    if results is None:
        print("⚠️ Truy vấn lỗi.")
        return

    print("\n📊 Dữ liệu:")
    print(results.head(5))

    conn.close()

# Thực thi khi chạy file
if __name__ == "__main__":
    question = "Lấy 10 dòng đâu tiên của bảng stocks."
    run_chat(question)


❌ Lỗi kết nối cơ sở dữ liệu: connection to server at "aws-0-ap-southeast-1.pooler.supabase.com" (52.77.146.31), port 5432 failed: SSL connection has been closed unexpectedly



In [14]:
question = "Lấy dữ liệu ngày 2023-01-01 đến 2024-01-10."
run_chat(question)

✅ Kết nối thành công đến PostgreSQL.

🧠 SQL sinh ra:
SELECT *
FROM stocks
WHERE Date BETWEEN '2023-01-01' AND '2024-01-10';

📊 Dữ liệu:
         date   price    open    high     low     vol change_percent
0  2024-01-10  186.19  184.35  186.40  183.92  45.90M          0.57%
1  2024-01-09  185.14  183.92  185.15  182.73  42.84M         -0.23%
2  2024-01-08  185.56  182.09  185.60  181.50  59.14M          2.42%
3  2024-01-05  181.18  181.99  182.76  180.17  62.38M         -0.40%
4  2024-01-04  181.91  182.15  183.09  180.88  70.70M         -1.27%


C:\Users\thong\AppData\Local\Temp\ipykernel_22992\2190399169.py:97: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)



💡 Trả lời từ AI:
❌ Lỗi sinh câu trả lời: Error code: 400 - {'error': {'message': "This model's maximum context length is 8192 tokens. However, your messages resulted in 8792 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}


In [15]:
question = "Tổng dữ liệu đóng cửa ngày 2023-01-01 đến 2024-01-10."
run_chat(question)

✅ Kết nối thành công đến PostgreSQL.

🧠 SQL sinh ra:
SELECT SUM(Price) 
FROM stocks 
WHERE Date BETWEEN '2023-01-01' AND '2024-01-10';

📊 Dữ liệu:
       sum
0  44427.1


C:\Users\thong\AppData\Local\Temp\ipykernel_22992\2190399169.py:97: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)



💡 Trả lời từ AI:
The total closing data from January 1, 2023 to January 10, 2024 is 44,427.1.


In [16]:
question = "Tổng dữ liệu đóng cửa ngày 2023-01-01 đến 2024-01-10. sau đó lấy giá trị đó chia cho 2."
run_chat(question)

✅ Kết nối thành công đến PostgreSQL.

🧠 SQL sinh ra:
SELECT SUM(Price) / 2 
FROM stocks 
WHERE Date BETWEEN '2023-01-01' AND '2024-01-10';

📊 Dữ liệu:
   ?column?
0  22213.55


C:\Users\thong\AppData\Local\Temp\ipykernel_22992\2190399169.py:97: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)



💡 Trả lời từ AI:
The total closing data from January 1, 2023 to January 10, 2024 is 22213.55. When this value is divided by 2, it will yield the final result.
